In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import csv
from math import *
import pickle
import itertools

from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity

from sklearn.grid_search import GridSearchCV

In [2]:
data = np.loadtxt('data-final/train-final.txt', dtype=int, delimiter=',',skiprows=1)
orig_data = np.copy(data)
num_sensors = data.shape[1]-1

In [3]:
def query_data(data, sensor, time_start, time_end):
    # Get starting index and ending index, derived from time_start and time_end
    st_day, en_day = time_start/10000, time_end/10000
    st_hour, en_hour = (time_start%10000)/100, (time_end%10000)/100
    st_min, en_min = time_start%100, time_end%100
    st_index = (st_day-1)*1440 + st_hour*60 + st_min
    en_index = (en_day-1)*1440 + en_hour*60 + en_min
    
    return np.sum(data[st_index:en_index+1,sensor])

#### Get nearest neighbors of each piece

In [4]:
with open('data-final/sensor-coordinates.txt') as f:
    coords = list()
    reader = csv.reader(f)
    reader.next()
    for row in reader:
        coords.append((float(row[1]), float(row[2])))

In [5]:
def dist(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

In [6]:
def get_nearest(coords, r=10):
    nearest = list()
    for i in range(56):
        temp = list()
        for j in range(56):
            if i != j:
                if dist(coords[i], coords[j]) < r:
                    temp.append(j+1)
        nearest.append((i+1, tuple(temp)))
    return nearest

In [7]:
n = get_nearest(coords,10)

In [8]:
timestamps = np.unique(data[:,0]%10000)

In [9]:
len(timestamps)

1440

#### Create intervals to sample over

In [ ]:
window_size = 20
intervals=[]
for ii in range(0,len(timestamps)-window_size+1,20):
    intervals.append(timestamps[ii:ii+window_size])

The idea is to estimate the distribution for each 20 minute interval (maybe less, 5 or 10 minutes might make more sense) for each sensor. 

From this distribution, you can randomly sample a specified number of times and then take the average of this sample to fill in the missing data. 

After you've filled the missing data you can then take a spatial average (using the nearest neighbors of each sensor that has missing data at that time step), giving the neighbors a higher weighting. 

Do this form of median filtering a couple of times. Check and see...

In [ ]:
sampled_flow_by_interval = np.zeros((len(intervals),num_sensors))
ival_count = 0
for intval in intervals:
    print ival_count
    for i_sens in range(num_sensors):
        sensor = data[:, [0, i_sens+1]]
        sensor = sensor[sensor[:, 1] > -1]
        sensor[:, 0] %= 10000
        X = list()
        for time in intval:
            filtered = sensor[sensor[:, 0] == time]
            if filtered.shape[0] != 0:
                choice = np.random.choice(np.arange(filtered.shape[0]))

                X.append(filtered[:,1])
        X = list(itertools.chain.from_iterable(X))
        
        grid = GridSearchCV(KernelDensity(),
                    {'bandwidth': np.logspace(-1, 1, 10)},
                    cv=20) # 20-fold cross-validation
        grid.fit(np.array(X).reshape(len(X), 1))
        kde = grid.best_estimator_
        
        # Sample from kde and find expected value
        sampled_entry = np.abs(np.ceil(np.mean(np.abs(kde.sample(250)))-0.8))
        
        # Place sampled entry into look-up matrix
        sampled_flow_by_interval[ival_count,i_sens] = sampled_entry
    ival_count += 1


0
1

#### Place interval sampling into missing data locations

In [ ]:
sensor_list = np.linspace(1,56,56)
for i in range(data.shape[0]):
    temp = data[i,1:]
    missing_sensors = sensor_list[temp==-1]
    if len(missing_sensors) == 0:
        continue

    time = data[i,0]%10000
    
    sample_intervals = []
    counter = 0
    for intval in intervals:
        if time in intval:
            sample_intervals.append(counter)
        counter += 1
        
    for i_sens in missing_sensors:
        data[i,int(i_sens-1)] = np.mean(sampled_flow_by_interval[sample_intervals,int(i_sens-1)])

#### Take spatial average twice (accounts for neighboring sensors being offline)

In [ ]:
for repeat in range(2):
    for i in range(data.shape[0]):
        temp = orig_data[i,1:]
        missing_sensors = sensor_list[temp==-1]
        if len(missing_sensors)==0:
            continue
        for i_sens in missing_sensors:
            neighbors = n[int(i_sens-1)][1]
            num_neighbors = len(neighbors)
            weighting = float(2*num_neighbors + 1)
        
            data[i,i_sens] = (2/weighting)*(np.sum(data[i,neighbors])) + data[i,int(i_sens)]/weighting
    

#### Query sampled data set and submit

In [ ]:
with open('data/test.txt') as csvfile:
    reader = csv.reader(csvfile)
    reader.next()
    with open ('final_submission.txt', 'w') as f:
        counter = 1
        f.write('Index,Count\n')
        for row in reader:
            f.write('{},{}\n'.format(counter, query_data(data,int(row[0][1:]),int(row[1]),int(row[2]))))
            counter += 1